# Export Bathymetry

Exports bathymetry tiles from a geometry of interest to GEE assets and GCS.
TODO: create logic in outside of notebook

In [1]:
from pathlib import Path
import sys
import time

import ee
import geemap
import geetools
import geojson

sys.path.append(str(Path.cwd().parent.parent))

from eobathymetry import bathymetry, tiler, ee_tiler, utils

## Create tiles based on input geometry

In [2]:
Map = geemap.Map(center=(52.97, 4.74), zoom=10)

aoi_json = """{
  "type": "Polygon",
  "coordinates": [
    [
      [
        4.726555204480136,
        52.79894952106581
      ],
      [
        5.382988309948886,
        53.2615577684405
      ],
      [
        5.226433134167636,
        53.48931215536743
      ],
      [
        4.770500516980136,
        53.41898585234949
      ],
      [
        4.270622587292636,
        52.91018589685636
      ],
      [
        4.726555204480136,
        52.79894952106581
      ]
    ]
  ]
}"""
bounds = ee.Geometry(geojson.loads(aoi_json))

## Plot tiles for quality control

In [3]:
print(tiler.to_radians(10))
print(ee_tiler.to_radians(10).getInfo())

0.17453292519943295
0.17453292519943295


In [4]:
print(tiler.zoom_to_scale(10))
print(ee_tiler.zoom_to_scale(10).getInfo())

152.87405657196044
152.87405657196044


In [5]:
print(tiler.degrees_to_tiles(10, 10, 10)[1])
print(ee_tiler.degrees_to_tiles(10, 10, 10).get(1).getInfo())

483
483


In [6]:
import math
print(math.pow(2, 10) - 30 - 1)
print(ee.Number(2).pow(ee.Number(10)).subtract(30).subtract(1).getInfo())

993.0
993


In [7]:
print(tiler.get_tile_bounds(10, 30, 10))
print(ee_tiler.get_tile_bounds(10, 30, 10).getInfo())

[[-19646150.757965025, 18824299.830255676], [-19607014.999482602, 18863435.588738095]]
[[-19646150.757965025, 18824299.830255676], [-19607014.999482602, 18863435.588738095]]


In [8]:
print(tiler.pixels_to_meters(1298379, 129387198, 11))
print(ee_tiler.pixels_to_meters(1298379, 129387198, 11).getInfo())

[79206724.00613347, 9869935405.026934]
[79206724.00613347, 9869935405.026934]


In [9]:
print(tiler.get_tiles_for_geometry(bounds, 10).size().getInfo())
print("------------------")
print(ee_tiler.get_tiles_for_geometry(bounds, 10).size().getInfo())

12
------------------
12


In [10]:
def add_tile_bounds(zoom):
    tiles = ee_tiler.get_tiles_for_geometry(bounds, zoom)
    Map.addLayer(tiles.style(width=max(1, 10 - zoom), fillColor= "00000022"), {}, "tiles " + str(zoom))

In [11]:
list(map(add_tile_bounds, [8, 9, 10, 11, 12]))
Map

Map(center=[52.97, 4.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

## Export to earth engine assets
Create tiles to export. Then export based tiles and given zoom levels.

In [12]:
from typing import List
import re
import geetools
def export_image_collection(image_collection: ee.ImageCollection, name_pattern: str, scale: int) -> List[ee.batch.Task]:
    def make_name(image: ee.Image, name_pattern: str) -> str:
        patterns: ee.List = ee.List(re.findall(r"\{(.*?)\}", name_pattern))
        props: ee.Dictionary = image.toDictionary()
        image_values: ee.List = patterns.map(lambda pattern: ee.Dictionary({pattern: ee.String(image.get(pattern))}))
        

In [50]:
from typing import List
from dateutil.parser import parse
from geetools import batch
import functools


def export_tile_bathymetry(tile: ee.Feature, start: ee.String, stop: ee.String, asset_path: str) -> ee.Image:
    bounds: ee.Geometry = ee.Feature(tile).geometry().bounds(1)
    scale: ee.Number = ee_tiler.zoom_to_scale(ee.Number.parse(tile.get("zoom")))
    sdb: Bathymetry = bathymetry.Bathymetry()
    zoom: ee.String = ee.String(tile.get("zoom"))
    tx: ee.String = ee.String(tile.get("tx"))
    ty: ee.String = ee.String(tile.get("ty"))
    tile_name: ee.String = ee.String("z").cat(zoom).cat("_x").cat(tx).cat("_y").cat(ty)
    tile_fullname: ee.String = ee.String(tile_name).cat("_t").cat(ee.String(ee.Date(start).millis()))
    tile.set(tile_name)
        
    image: ee.Image = sdb.compute_inverse_depth(
                bounds=bounds,
                start=start,
                stop=stop,
                filter_masked=True,
                scale=ee_tiler.zoom_to_scale(ee.Number.parse(tile.get("zoom"))).multiply(5)
    )
    image = image.set(
        "system:time_start", ee.Date(start).millis(),
        "system:time_stop", ee.Date(stop).millis(),
        "zoom", ee.Number.parse(zoom),
        "tx", ee.Number.parse(tx),
        "ty", ee.Number.parse(ty)
    )
    ee.batch.Export.image.toAsset(
        image,
        description="export SDB tiles",
        assetId=ee.String(asset_path).cat(tile_fullname),
#         pyramidingPolicy=None,
        dimensions=None,
        region=bounds,
        scale=None,
        crs=None,
        crsTransform=None,
        maxPixels=None
    ) 

def export_tiles_to_assets(
    asset_path: str,
    geometry: ee.Geometry,
    sdb: bathymetry.Bathymetry,
    zoom: int,
    start: str,
    stop: str,
    step_months: int = 3,
    window_years: int = 2,
#     preview: bool = False
):
    
    def create_year_window(year: ee.Number, month: ee.Number) -> ee.Dictionary:
        t: ee.Date = ee.Date.fromYMD(year, month, 1)
        d_format: str = "YYYY-MM-dd"
        return ee.Dictionary({"start": t.format(d_format), "stop": t.advance(window_years, 'year').format(d_format)})
        
    dates: ee.List = ee.List.sequence(parse(start).year, parse(stop).year).map(
        lambda year: ee.List.sequence(1, 12, step_months).map(functools.partial(create_year_window, year))
    ).flatten()
    
    tiles: ee.FeatureCollection = ee_tiler.get_tiles_for_geometry(geometry, ee.Number(zoom))
#     buffered_bounds: ee.Geometry = tiles.first().geometry().buffer(ee_tiler.zoom_to_scale(zoom)).bounds()
#     Map.addLayer(get_tile_bathymetry(tiles.first(), start, stop).clip(buffered_bounds), {}, "preview SDB")
    dates.map(
        lambda date: tiles.map(
            lambda tile: export_tile_bathymetry(tile, ee.String(start), ee.String(stop), asset_path))
    )
#     print(bathymetry_tiles.first().getInfo())
#     batch.imagecollection.toAsset(
#         bathymetry_tiles,
#         asset_path,
#         namePattern=None,
#         scale=ee_tiler.zoom_to_scale(ee.Number(zoom)).getInfo(),
#         region=None,
#         create=True,
#         verbose=False,
#         datePattern='yyyyMMdd',
#         extra=None
#     )

In [51]:
user_name = ee.data.getAssetRoots()[0]["id"].split("/")[-1]
asset_id = f"users/{user_name}/eo-bathymetry"
asset_dict = ee.data.listAssets({"parent": f"projects/earthengine-legacy/assets/users/{user_name}"})
if any([f"users/{user_name}/eo-bathymetry" in asset["id"] for asset in asset_dict["assets"]]):
    ee.data.deleteAsset(asset_id)
# ee.data.create_assets(asset_ids=["/".join(asset_id.split("/"))], asset_type="Folder", mk_parents=True)

export_tiles_to_assets(asset_id, bounds, bathymetry.Bathymetry(), 10, "2020-01-01", "2021-01-01")

TypeError: expected string or bytes-like object